## Financial News Sentiment Production

Given financial news headlines, let's try to predict the sentiment of a given headline.

Data source: https://www.kaggle.com/datasets/ankurzing/sentiment-analysis-for-financial-news

We will use a Tensorflow RNN to make our predictions. 

### Getting Started

In [1]:
import numpy as np
import pandas as pd

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split

import tensorflow as tf

2024-08-11 12:12:42.231659: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2 AVX AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [4]:
data = pd.read_csv('all-data.csv', names=['Label', 'Text'],  encoding='latin-1')
data

,Label,Text
0,neutral,"According to Gran , the company has no plans t..."
1,neutral,Technopolis plans to develop in stages an area...
2,negative,The international electronic industry company ...
3,positive,With the new production plant the company woul...
4,positive,According to the company 's updated strategy f...
...,...,...
4841,negative,LONDON MarketWatch -- Share prices ended lower...
4842,neutral,Rinkuskiai 's beer sales fell by 6.5 per cent ...
4843,negative,Operating profit fell to EUR 35.4 mn from EUR ...
4844,negative,Net sales of the Paper segment decreased to EU...


In [5]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4846 entries, 0 to 4845
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Label   4846 non-null   object
 1   Text    4846 non-null   object
dtypes: object(2)
memory usage: 75.8+ KB


### Preprocessing

In [8]:
data['Label'].unique()

array(['neutral', 'negative', 'positive'], dtype=object)

In [21]:
def get_sequences(texts):
    tokenizer = Tokenizer()
    tokenizer.fit_on_texts(texts)
    
    sequences = tokenizer.texts_to_sequences(texts)
    print("Vocab Length:", len(tokenizer.word_index) + 1)
    
    max_seq_length = np.max(list(map(lambda x: len(x), sequences)))
    print('Maximum sequence length:', max_seq_length)
    
    sequences = pad_sequences(sequences, maxlen = max_seq_length, padding='post')
    
    return sequences

In [24]:
get_sequences(data['Text']), get_sequences(data['Text']).shape

Vocab Length: 10123
Maximum sequence length: 71
Vocab Length: 10123
Maximum sequence length: 71


(array([[  94,    5, 3498, ...,    0,    0,    0],
        [ 840,  336,    5, ...,    0,    0,    0],
        [   1,  293,  656, ...,    0,    0,    0],
        ...,
        [  42,   31,  242, ...,    0,    0,    0],
        [  30,   27,    2, ...,    0,    0,    0],
        [  27,    3,   35, ...,    0,    0,    0]], dtype=int32),
 (4846, 71))

In [25]:
def preprocess_inputs(df):
    df = df.copy()

    sequences = get_sequences(df['Text'])

    label_mapping = {
        'negative': 0,
        'neutral': 1,
        'positive': 2
    }

    y = df['Label'].replace(label_mapping)

    train_sequences, test_sequences, y_train, y_test = train_test_split(sequences, y, train_size=0.7, shuffle=True, random_state=1)
    
    return train_sequences, test_sequences, y_train, y_test

In [26]:
train_sequences, test_sequences, y_train, y_test = preprocess_inputs(data)
train_sequences

Vocab Length: 10123
Maximum sequence length: 71


array([[5442,  510,   16, ...,    0,    0,    0],
       [  22, 1628,    4, ...,    0,    0,    0],
       [1141,  936,  136, ...,    0,    0,    0],
       ...,
       [   1,  419,   16, ...,    0,    0,    0],
       [2586,  123, 3247, ...,    0,    0,    0],
       [  30,  615,  555, ...,    0,    0,    0]], dtype=int32)

In [27]:
y_train

545     2
2374    0
4217    1
1071    1
716     2
       ..
2895    1
2763    1
905     2
3980    1
235     2
Name: Label, Length: 3392, dtype: int64

### Training

In [33]:
inputs = tf.keras.Input(shape=(train_sequences.shape[1],))
x = tf.keras.layers.Embedding(
    input_dim=10123,
    output_dim=128,
    input_length=train_sequences.shape[1]
)(inputs)
x = tf.keras.layers.GRU(256, return_sequences=True, activation='tanh')(x)
x = tf.keras.layers.Flatten()(x)
outputs = tf.keras.layers.Dense(3, activation='softmax')(x)

model = tf.keras.Model(inputs=inputs, outputs=outputs)

model.compile(
    optimizer='adam',
    loss = 'sparse_categorical_crossentropy',
    metrics = ['accuracy']
)

history = model.fit(
    train_sequences,
    y_train,
    validation_split=0.2,
    batch_size=32,
    epochs=100,
    callbacks=[
        tf.keras.callbacks.EarlyStopping(
            monitor='val_loss',
            patience=3,
            restore_best_weights=True
        )
    ]
)

Epoch 1/100


2024-08-11 13:51:14.072756: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2024-08-11 13:51:14.075561: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2024-08-11 13:51:14.077603: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

85/85 [==============================] - ETA: 0s - loss: 0.8310 - accuracy: 0.6439

2024-08-11 13:51:32.069809: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2024-08-11 13:51:32.071488: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2024-08-11 13:51:32.072872: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

85/85 [==============================] - 19s 201ms/step - loss: 0.8310 - accuracy: 0.6439 - val_loss: 0.8336 - val_accuracy: 0.6421
Epoch 2/100
85/85 [==============================] - 16s 189ms/step - loss: 0.4929 - accuracy: 0.7965 - val_loss: 0.7372 - val_accuracy: 0.6996
Epoch 3/100
85/85 [==============================] - 17s 197ms/step - loss: 0.1575 - accuracy: 0.9469 - val_loss: 0.8948 - val_accuracy: 0.7187
Epoch 4/100
85/85 [==============================] - 16s 193ms/step - loss: 0.0428 - accuracy: 0.9856 - val_loss: 1.1644 - val_accuracy: 0.7010
Epoch 5/100
85/85 [==============================] - 17s 202ms/step - loss: 0.0226 - accuracy: 0.9948 - val_loss: 1.2009 - val_accuracy: 0.7069


### Results

In [34]:
model.evaluate(test_sequences, y_test)

46/46 [==============================] - 3s 66ms/step - loss: 0.6658 - accuracy: 0.7407


[0.6657790541648865, 0.7407152652740479]

1454